In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import TextStreamer
import torch

device = "cuda" # the device to load the model onto
model_path = "Qwen/Qwen2.5-3B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
  model_path,
  torch_dtype=torch.bfloat16, 
  device_map=device,
  attn_implementation="flash_attention_2"
)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code = True)

# prepare messages to model




Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
messages = [
    {"role": "system", "content": "Bạn là trợ lý tiếng Việt hữu ích. Hãy trả lời người dùng một cách chính xác."},
]


In [9]:

# prompt = """Hãy cho bản thân không gian để suy nghĩ bằng cách trích xuất các đoạn văn có liên quan từ ngữ cảnh dưới đây trước khi trả lời câu hỏi của người dùng.

# Sử dụng các đoạn ngữ cảnh sau:

# * Nội dung của Tiểu dự án 1, Dự án 4 là gì? "Nội dung của Tiểu dự án 1, Dự án 4 thuộc Chương trình mục tiêu quốc gia phát triển kinh tế - xã hội vùng đồng bào dân tộc thiểu số và miền núi:
# - Nội dung số 01: Đầu tư cơ sở hạ tầng thiết yếu vùng đồng bào dân tộc thiểu số và miền núi; ưu tiên đối với các xã đặc biệt khó khăn, thôn đặc biệt khó khăn.
# + Đầu tư xây dựng, cải tạo các công trình giao thông nông thôn phục vụ sản xuất, kinh doanh và dân sinh; công trình cung cấp điện phục vụ sinh hoạt và sản xuất, kinh doanh trên địa bàn thôn, bản; trạm chuyển tiếp phát thanh xã, nhà sinh hoạt cộng đồng; trường, lớp học đạt chuẩn; các công trình thủy lợi nhỏ; các công trình hạ tầng quy mô nhỏ khác do cộng đồng đề xuất, phù hợp với phong tục, tập quán để phục vụ nhu cầu của cộng đồng, phù hợp với mục tiêu của Chương trình và quy định của pháp luật; ưu tiên công trình có nhiều hộ nghèo, phụ nữ hưởng lợi;
# + Đầu tư xây dựng, nâng cấp, cải tạo, sửa chữa, bảo dưỡng, mua sắm trang thiết bị cho các trạm y tế xã bảo đảm đạt chuẩn;
# + Đầu tư cứng hóa đường đến trung tâm xã chưa được cứng hóa; ưu tiên đầu tư đối với các xã chưa có đường từ trung tâm huyện đến trung tâm xã, đường liên xã (từ trung tâm xã đến trung tâm xã);
# + Đầu tư cơ sở hạ tầng trọng điểm kết nối các xã đặc biệt khó khăn trên cùng địa bàn (hệ thống hạ tầng phục vụ giáo dục, y tế; hệ thống cầu, đường giao thông; hạ tầng lưới điện...); xây dựng cầu dân sinh để phục vụ sinh hoạt, tăng cường kết nối, tạo trục động lực phát triển đồng bộ trên cơ sở thúc đẩy liên kết giữa các xã đặc biệt khó khăn nhằm phát huy sức mạnh tiểu vùng giúp phát triển bền vững và góp phần xây dựng nông thôn mới vùng đồng bào dân tộc thiểu số và miền núi;
# + Đầu tư, hỗ trợ kinh phí xây dựng thí điểm 04 nhà hỏa táng điện cho đồng bào dân tộc thiểu số tỉnh Ninh Thuận, Bình Thuận (mỗi tỉnh 02 công trình);
# + Duy tu, bảo dưỡng công trình cơ sở hạ tầng trên địa bàn đặc biệt khó khăn và công trình cơ sở hạ tầng các xã, thôn đã đầu tư từ giai đoạn trước.
# - Nội dung số 02: Đầu tư xây dựng, cải tạo nâng cấp mạng lưới chợ vùng đồng bào dân tộc thiểu số và miền núi."
#  (Theo Phần III, Mục 4 Quyết định Thủ tướng chính phủ 1719/QĐ-TTg)

# * Mục tiêu của Nội dung 1, Tiểu dự án 1, Dự án 10 là gì? "Mục tiêu của Nội dung số 01, Tiểu dự án 1, Dự án 10 là: 
# Xây dựng, nâng cao chất lượng và hiệu quả công tác vận động, phát huy vai trò của lực lượng cốt cán và người có uy tín trong vùng đồng bào dân tộc thiểu số và miền núi. Biểu dương, tôn vinh, ghi nhận công lao, sự đóng góp của các điển hình tiên tiến trong vùng đồng bào dân tộc thiểu số và miền núi trong sự nghiệp xây dựng, bảo vệ Tổ quốc và hội nhập quốc tế."
#  (Theo Phần III, Mục 10 Quyết định Thủ tướng chính phủ 1719/QĐ-TTg)

# * Nội dung của Tiểu dự án 1, Dự án 5 là gì? "Nội dung của Tiểu dự án 1, Dự án 5 thuộc Chương trình mục tiêu quốc gia phát triển kinh tế - xã hội vùng đồng bào dân tộc thiểu số và miền núi:
# + Đầu tư cơ sở vật chất, trang thiết bị cho các trường phổ thông dân tộc nội trú, bán trú, có học sinh bán trú:
# . Nâng cấp, cải tạo cơ sở vật chất khối phòng/công trình phục vụ ăn, ở, sinh hoạt cho học sinh và phòng công vụ giáo viên;
# . Nâng cấp, cải tạo cơ sở vật chất/khối phòng/công trình phục vụ học tập; bổ sung, nâng cấp các công trình phụ trợ khác;
# . Đầu tư cơ sở vật chất phục vụ chuyển đổi số giáo dục phục vụ việc giảng dạy và học tập trực tuyến cho học sinh dân tộc thiểu số;
# . Ưu tiên đầu tư xây dựng trường dân tộc nội trú cho huyện có đông đồng bào dân tộc thiểu số sinh sống nhưng chưa có hoặc phải đi thuê địa điểm để tổ chức hoạt động.
# + Xóa mù chữ cho người dân vùng đồng bào dân tộc thiểu số:
# . Xây dựng tài liệu phục vụ hướng dẫn dạy xoá mù chữ, thiết kế công nghệ, thiết bị lưu trữ cơ sở dữ liệu về xoá mù chữ, dạy học xoá mù chữ;
# . Bồi dưỡng, tập huấn, truyền thông, tuyên truyền;
# . Hỗ trợ người dân tham gia học xoá mù chữ;
# . Hỗ trợ tài liệu học tập, sách giáo khoa, văn phòng phẩm."
#  (Theo Phần III, Mục 5 Quyết định Thủ tướng chính phủ 1719/QĐ-TTg)

# * Nội dung cụ thể của Dự án 1 là gì? "Nội dung cụ thể của Dự án 1 bao gồm:
# - Nội dung số 01: Hỗ trợ đất ở: Căn cứ quỹ đất, hạn mức đất ở và khả năng ngân sách, Ủy ban nhân dân cấp tỉnh xem xét, quyết định giao đất để làm nhà ở cho các đối tượng nêu trên phù hợp với điều kiện, tập quán ở địa phương và pháp luật về đất đai, cụ thể:
# + Ở những nơi có điều kiện về đất đai, chính quyền địa phương sử dụng số tiền hỗ trợ từ ngân sách để tạo mặt bằng, làm hạ tầng kỹ thuật để cấp đất ở cho các đối tượng được thụ hưởng;
# + Ở các địa phương không có điều kiện về đất đai, chính quyền địa phương bố trí kinh phí hỗ trợ cho người dân tự ổn định chỗ ở theo hình thức xen ghép.
# - Nội dung số 02: Hỗ trợ nhà ở: Hỗ trợ xây dựng 01 căn nhà theo phong tục tập quán của địa phương, định mức tính theo xây dựng 01 căn nhà cấp 4 đảm bảo 3 cứng (nền cứng, khung - tường cứng, mái cứng).
# - Nội dung số 03: Hỗ trợ đất sản xuất, chuyển đổi nghề: Hộ dân tộc thiểu số nghèo; hộ nghèo dân tộc Kinh sinh sống ở xã đặc biệt khó khăn, thôn đặc biệt khó khăn vùng đồng bào dân tộc thiểu số và miền núi làm nghề nông, lâm, ngư nghiệp không có hoặc thiếu từ 50% đất sản xuất trở lên theo định mức của địa phương thì được hưởng một trong hai chính sách sau:
# + Hỗ trợ trực tiếp đất sản xuất: Hộ không có đất sản xuất nếu có nhu cầu thì được chính quyền địa phương trực tiếp giao đất sản xuất;
# + Hỗ trợ chuyển đổi nghề: Trường hợp chính quyền địa phương không bố trí được đất sản xuất thì hộ không có đất hoặc thiếu đất sản xuất được hỗ trợ chuyển đổi nghề.
# - Nội dung số 04: Hỗ trợ nước sinh hoạt:
# + Hỗ trợ nước sinh hoạt phân tán: Ưu tiên hỗ trợ để mua sắm trang bị hoặc xây dựng bể chứa nước phục vụ sinh hoạt của hộ gia đình;
# + Hỗ trợ nước sinh hoạt tập trung: Đầu tư xây dựng công trình nước tập trung theo dự án được cấp có thẩm quyền phê duyệt. Ưu tiên cho người dân vùng thường xuyên xảy ra hạn hán, xâm nhập mặn, vùng đặc biệt khó khăn, vùng cao chưa có nguồn nước hoặc thiếu nước sinh hoạt hợp vệ sinh.
# - Hộ gia đình thuộc diện đối tượng theo quy định của Dự án này có nhu cầu vay vốn được vay từ Ngân hàng Chính sách xã hội để có đất ở, xây dựng mới hoặc sửa chữa nhà ở, tạo quỹ đất sản xuất, học nghề và chuyển đổi nghề." (Theo Phần III, Mục 1 Quyết định Thủ tướng chính phủ 1719/QĐ-TTg)

# * Nội dung của Dự án 2 là gì? "Nội dung của Dự án 2 thuộc Chương trình mục tiêu quốc gia phát triển kinh tế - xã hội vùng đồng bào dân tộc thiểu số và miền núi:
# - Hỗ trợ khảo sát vị trí, địa điểm phục vụ công tác quy hoạch, lập, thẩm định, phê duyệt dự án đầu tư bố trí ổn định dân cư.
# - Hỗ trợ đầu tư xây dựng kết cấu hạ tầng:
# + Bồi thường, giải phóng mặt bằng, san lấp mặt bằng đất ở tại điểm tái định cư;
# + Khai hoang đất sản xuất;
# + Đầu tư xây dựng: Đường giao thông (nội vùng dự án và đường nối điểm dân cư mới đến tuyến giao thông gần nhất); công trình thủy lợi nhỏ, điện, nước sinh hoạt và một số công trình thiết yếu khác.
# - Hỗ trợ trực tiếp hộ gia đình:
# + Hỗ trợ nhà ở, đất sản xuất (đối với trường hợp phải thay đổi chỗ ở);
# + Hỗ trợ kinh phí di chuyển các hộ từ nơi ở cũ đến nơi tái định cư;
# + Thực hiện các chính sách hỗ trợ hiện hành khác đối với các hộ được bố trí ổn định như người dân tại chỗ.
# - Hỗ trợ địa bàn bố trí dân xen ghép:
# + Điều chỉnh đất ở, đất sản xuất giao cho các hộ mới đến (khai hoang, bồi thường theo quy định khi thu hồi đất của các tổ chức, cá nhân khi thu hồi đất);
# + Xây mới hoặc nâng cấp lớp học, trạm y tế, các công trình thủy lợi nội đồng, đường dân sinh, điện, nước sinh hoạt và một số công trình hạ tầng thiết yếu khác." (Theo Phần III, Mục 1 Quyết định Thủ tướng chính phủ 1719/QĐ-TTg)

# Các đoạn văn có liên quan: <trích xuất các đoạn văn có liên quan từ ngữ cảnh tại đây>
# Câu hỏi: Nội dung dự án 1 là gì?
# """
prompt = "Đặc sản của Thanh Hóa là gì vậy?"
messages.append({"role": "user", "content": prompt})
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(device)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
generated_ids = model.generate(model_inputs.input_ids, 
                            do_sample=True,
                            temperature=0.1,  
                                top_k=40,  
                                top_p=0.95,  
                                max_new_tokens=1024,
                                repetition_penalty = 1.05,  
                            streamer=streamer)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

messages.append({"role": "assistant", "content": response})

Thanh Hóa nổi tiếng với nhiều đặc sản nổi tiếng như:

1. Bánh bèo tôm cá: Một món ăn dân dã nhưng rất ngon, được làm từ bột gạo nếp, tôm và cá.

2. Bánh mì Thanh Hóa: Loại bánh mì đặc trưng với vị chua ngọt đặc trưng.

3. Mắm tôm: Một loại mắm truyền thống có hương vị đặc trưng.

4. Bánh đa cua: Món ăn nổi tiếng với cua đồng hấp dẫn.

5. Bánh tráng nướng: Một món ăn vặt phổ biến, thường được ăn kèm với thịt heo xé.

6. Bánh canh cua đồng: Món ăn ngon miệng với cua đồng nấu cùng với bánh canh.

7. Mì Quảng Thanh Hóa: Loại mì đặc trưng với vị chua ngọt đặc trưng.

8. Bánh phu thê: Món bánh đặc sản với nhân đậu xanh và dừa.

9. Bánh đa cua đồng: Món ăn nổi tiếng với cua đồng hấp dẫn.

10. Mì Quảng: Loại mì đặc trưng với vị chua ngọt đặc trưng.

Những đặc sản này không chỉ mang lại hương vị đặc biệt mà còn góp phần tạo nên bản sắc văn hóa ẩm thực của vùng đất Thanh Hóa.
